# 第7章 文本数据

In [2]:
import numpy as np
import pandas as pd

## 一、string类型的性质
### 1. string与object的区别
#### string类型和object不同之处有三：
##### ①.字符存取方法（string accessor methods，如str.count）会返回相应数据的Nullable类型，而object会随缺失值的存在而改变返回类型
##### ②.某些Series方法不能在string上使用，例如：Series.str.decode()，因为存储的是字符串而不是字节
##### ③. string类型在缺失值存储或运算时，类型会广播为pd.NA，而不是浮点型np.nan

### 2.string类型的转换
#### 如果将一个其他类的容器直接转换为string类型，可能会出错

In [4]:
pd.Series([1,'1.']).astype('string')

ValueError: StringArray requires a sequence of strings or pandas.NA

In [7]:
pd.Series([True,False]).astype('string')

ValueError: StringArray requires a sequence of strings or pandas.NA

#### 当下正确的方法是分两步转换，先转为str型object，再转为string类型：

In [5]:
pd.Series([1,'1.']).astype('str')

0     1
1    1.
dtype: object

In [6]:
pd.Series([1,'1.']).astype('str').astype('string')

0     1
1    1.
dtype: string

In [8]:
pd.Series([True,False]).astype('str').astype('string')

0     True
1    False
dtype: string

In [10]:
a = pd.Series([True,False])
a

0     True
1    False
dtype: bool

In [12]:
a.dtype

dtype('bool')

In [13]:
pd.Series([True,False]).dtype

dtype('bool')

## 二、拆分与拼接
### 1.  str.split方法
#### （a）分割符与str的位置元素选取

In [14]:
s = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'], dtype="string")
s

0    a_b_c
1    c_d_e
2     <NA>
3    f_g_h
dtype: string

In [15]:
s.str.split('_')

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

##### 这里需要注意的是split后的类型为object，因为现在Series中的元素已经不是string，而包含了list，且string类型只能含有字符串

##### 对于str方法可以进行元素的选择，如果该单元格元素是列表，那么str[i]表示取出第i个元素，如果是单个元素，则先把元素转为列表list再取出

In [16]:
s.str.split('_').str[1]

0       b
1       d
2    <NA>
3       g
dtype: object

In [17]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object").str[1]

0    _
1    b
dtype: object

#### (b) 其他参数
##### expend参数控制了是否将列拆开，n参数代表了最多分隔几次

In [18]:
s.str.split('_')

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [19]:
s.str.split('_',expand = True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


In [20]:
s.str.split('_',n = 1)

0    [a, b_c]
1    [c, d_e]
2        <NA>
3    [f, g_h]
dtype: object

In [21]:
s.str.split('_',expand = True,n = 1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


### 2. str.cat方法
#### （a） 不同对象的拼接方法
##### cat方法对于不同对象的作用结果并不相同，其中的对象包括：单列、双列、多列

##### ①.对于单个Series而言，就是指所有的元素进行字符合并为一个字符串

In [22]:
s = pd.Series(['ab',None,'d'],dtype = 'string')
s

0      ab
1    <NA>
2       d
dtype: string

In [24]:
s.str.cat()

'abd'

##### 其中可选sep分隔符参数，和缺失值替代字符na_rep参数

In [25]:
s.str.cat(sep = '，')

'ab，d'

In [26]:
s.str.cat(sep = ',',na_rep= '*')

'ab,*,d'

##### ② 对于两个Series合并而言，是对应索引的元素进行合并

In [27]:
s2 = pd.Series(['24',None,None],dtype='string')
s2

0      24
1    <NA>
2    <NA>
dtype: string

In [30]:
s.str.cat(s2)  # 只要对应索引有一个为<NA>，则s.str.cat()该对应索引的结果就位<NA>

0    ab24
1    <NA>
2    <NA>
dtype: string

同样也有相应的参数，需要注意的是两个缺失值会被同时替换

In [31]:
s.str.cat(s2,sep = ',',na_rep='*')

0    ab,24
1      *,*
2      d,*
dtype: string

③ 多列拼接可以分为表的拼接和多Series拼接

##### 表的拼接

In [32]:
df = pd.DataFrame({0:['1','3','5'],1:['5','b',None]},dtype='string')
df

,0,1
0,1,5
1,3,b
2,5,<NA>


In [33]:
s.str.cat(df)

0    ab15
1    <NA>
2    <NA>
dtype: string

In [34]:
s.str.cat(pd.DataFrame({0:['1','3','5'],1:['5','b',None]},dtype='string'),na_rep='*')

0    ab15
1     *3b
2     d5*
dtype: string

##### 多个Series的拼接

In [35]:
s+'0'

0     ab0
1    <NA>
2      d0
dtype: string

In [36]:
s*2

0    abab
1    <NA>
2      dd
dtype: string

In [38]:
s.str.cat([s+'0',s*2])

0    abab0abab
1         <NA>
2        dd0dd
dtype: string

#### (b)cat中的索引对齐
#####  当前版本中，如果两边合并的索引不相同且未指定join参数，默认为左连接，设置join='left'

In [39]:
s2 = pd.Series(list('abc'),index = [1,2,3],dtype='string')
s2

1    a
2    b
3    c
dtype: string

In [42]:
s

0      ab
1    <NA>
2       d
dtype: string

In [43]:
s.str.cat(s2)

0    <NA>
1    <NA>
2      db
dtype: string

In [44]:
s.str.cat(s2,na_rep= '*')

0    ab*
1     *a
2     db
dtype: string

## 三、替换
##### 广义上的替换，就是指str.replace函数的应用，fillna是针对缺失值的替换

### 1. str.replace的常见用法

In [47]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

##### 第一个值写r开头的正则表达式，后一个写替换的字符串

In [48]:
s.str.replace(r'^[AB]','***')  #匹配以A 或 B 开头的

0       ***
1       ***
2         C
3    ***aba
4    ***aca
5          
6      <NA>
7      CABA
8       dog
9       cat
dtype: string

### 2.子组与函数替换
##### 通过正整数调用子组（0返回字符本身，从1开始才是子组）

In [49]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'*') # 取出以A、B、C开头的 后面的字符串[1：]切片 + '*' 

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

##### 利用?P<....>表达式可以对子组命名调用

In [51]:
s.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:x.group('two')[1:]+'*')

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

### 3. 关于str.replace 的注意事项
##### 首先，要明确str.replace和replace并不是一个东西：
##### str.replace针对的是object类型或string类型，默认是以正则表达式为操作，目前暂时不支持DataFrame上使用
##### replace针对的是任意类型的序列或数据框，如果要以正则表达式替换，需要设置regex=True，该方法通过字典可支持多列替换

##### (a) str.replace赋值参数不得为pd.NA

In [52]:
pd.Series(['A','B'],dtype='string').str.replace(r'[A]',pd.NA)

TypeError: repl must be a string or callable

In [60]:
pd.Series(['A','B'],dtype='O').str.replace(r'[A]',pd.NA)

TypeError: repl must be a string or callable

#####  此时，可以先转换为object类型再转换回来：

In [57]:
pd.Series(['A','B'],dtype='string').astype('O').replace(r'[A]',pd.NA,regex = True).astype('string') # 注意 没有了str！！

0    <NA>
1       B
dtype: string

In [58]:
pd.Series(['A','B'],dtype='O').replace(r'[A]',pd.NA,regex = True).astype('string')

0    <NA>
1       B
dtype: string

##### 至于为什么不用replace函数的regex替换（但string类型replace的非正则替换是可以的），原因在下面一条

#### （b）对于string类型Series，在使用replace函数时不能使用正则表达式替换

In [62]:
pd.Series(['A','B'],dtype='string').replace(r'[A]','C',regex=True)  #使用了regex 没有replace成功

0    A
1    B
dtype: string

In [64]:
pd.Series(['A','B'],dtype='O').replace(r'[A]','C',regex=True)   #object 类型 regex的replace 没问题

0    C
1    B
dtype: object

#### (c) string类型序列如果存在缺失值，不能使用replace

In [74]:
pd.Series(['A',np.nan],dtype='string').replace('A','B')

0       B
1    <NA>
dtype: string

In [67]:
pd.Series(['A',np.nan],dtype='string').str.replace('A','B')

0       B
1    <NA>
dtype: string

##### 综上，概况的说，除非需要赋值元素为缺失值（转为object再转回来），否则请使用str.replace方法

## 四、子串匹配与提取
### 1 .str.extract方法
#### （a）常见用法

In [75]:
pd.Series(['10-87', '10-88', '10-89'],dtype="string").str.extract(r'([\d]{2})-([\d]{2})')

,0,1
0,10,87
1,10,88
2,10,89


#####  使用子组名作为列名

In [82]:
pd.Series(['10-87', '10-88', '-89'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})-(?P<name_2>[\d]{2})') #没有匹配e成功，全为<NA>

,name_1,name_2
0,10,87
1,10,88
2,<NA>,<NA>


#####  利用?正则标记选择部分提取   ? 表示0-1次，非贪婪匹配，通常匹配零次

In [84]:
pd.Series(['10-87', '10-88', '-89'],dtype="string").str.extract(r'(?P<name1>[\d]{2})?-(?P<name2>[\d]{2})')

,name1,name2
0,10,87
1,10,88
2,<NA>,89


In [85]:
pd.Series(['10-87', '10-88', '10-'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})-(?P<name_2>[\d]{2})?')

,name_1,name_2
0,10,87
1,10,88
2,10,<NA>


#### (b) expand参数（默认为True） 
##### 对于一个子组的Series，如果expand设置为False，则返回Series，若大于一个子组，则expand参数无效，全部返回DataFrame
##### 对于一个子组的Index，如果expand设置为False，则返回提取后的Index，若大于一个子组且expand为False，报错

In [87]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="string")
s

A11    a1
B22    b2
C33    c3
dtype: string

In [88]:
s.index

Index(['A11', 'B22', 'C33'], dtype='object')

In [89]:
s.str.extract(r'([\w])')

,0
A11,a
B22,b
C33,c


In [90]:
s.str.extract(r'([\w])',expand = False)

A11    a
B22    b
C33    c
dtype: string

In [92]:
s.index.str.extract(r'([\w])')

,0
0,A
1,B
2,C


In [93]:
s.index.str.extract(r'([\w])',expand=False)

Index(['A', 'B', 'C'], dtype='object')

In [94]:
s.index.str.extract(r'([\w])([\d])')

,0,1
0,A,1
1,B,2
2,C,3


In [95]:
s.index.str.extract(r'([\w])([\d])',expand = False)

ValueError: only one regex group is supported with Index

### 2.str.extractall方法
#####  与extract只匹配第一个符合条件的表达式不同，extractall会找出所有符合条件的字符串，并建立多级索引（即使只找到一个！！！）

In [3]:
s = pd.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"],dtype="string")
two_groups = '(?P<letter>[a-z])(?P<digit>[0-9])'
s.str.extract(two_groups, expand=True)

,letter,digit
A,a,1
B,b,1
C,c,1


In [4]:
s.str.extractall(two_groups)

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
C 0          c     1

In [5]:
s['A'] = 'a1'
s.str.extractall(two_groups)

,,letter,digit
,match,,
A,0,a,1
B,0,b,1
C,0,c,1


##### 如果想查看第 i 层匹配，可使用xs方法

In [6]:
s = pd.Series(['a1a2','b1b2','c1c2'],index = ['A','B','C'],dtype='string')
s.str.extractall(two_groups)

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
  1          b     2
C 0          c     1
  1          c     2

In [8]:
s.str.extractall(two_groups).xs(1,level = 'match') # 查看第1层的匹配，level =‘match’不能忘，表示确定i的位置

,letter,digit
A,a,2
B,b,2
C,c,2


### 3. str.contains和str.match

##### 前者的作用为检测是否包含某种正则模式

In [9]:
pd.Series(['1',None,'3a','3b','03c'],dtype='string').str.contains(r'[0-9][a-z]')

0    False
1     <NA>
2     True
3     True
4     True
dtype: boolean

##### 可选参数为na

In [10]:
pd.Series(['1',None,'3a','3b','03c'],dtype='string').str.contains(r'[0-9][a-z]',na = False)

0    False
1    False
2     True
3     True
4     True
dtype: boolean

##### str.match与其区别在于，match依赖于python的re.match，检测内容为是否从头开始！！包含该正则模式

In [11]:
pd.Series(['1',None,'3a','3b','03c'],dtype='string').str.match(r'[0-9][a-z]',na = False)

0    False
1    False
2     True
3     True
4    False
dtype: boolean

In [12]:
pd.Series(['1',None,'3a','_3b','03c'],dtype='string').str.match(r'[0-9][a-z]',na = False)

0    False
1    False
2     True
3    False
4    False
dtype: boolean

## 五、常用字符串方法
### 1.过滤型方法
#### （a） str.strip
##### 常用于过滤空格

In [13]:
pd.Series(list('abc'),index=[' space1  ','space2  ','  space3'],dtype="string")

 space1      a
space2       b
  space3     c
dtype: string

In [14]:
pd.Series(list('abc'),index=[' space1  ','space2  ','  space3'],dtype="string").index.str.strip()

Index(['space1', 'space2', 'space3'], dtype='object')

#### (b)str.lower和str.upper

In [15]:
pd.Series('A',dtype = 'string').str.lower()

0    a
dtype: string

In [16]:
pd.Series('a',dtype = 'string').str.upper()

0    A
dtype: string

#### (c) str.swapcase和str.capitalize
##### 分别表示交换字母大小写和大写首字母(其他字母变小写)

In [19]:
pd.Series('abCD',dtype = 'string').str.swapcase()

0    ABcd
dtype: string

In [25]:
pd.Series('aBCD',dtype = 'string').str.capitalize()  # str.capitalize() 首字母大写。其它字母变小写

0    Abcd
dtype: string

### 2.isnumeric方法
##### 检查每一位是否都是数字，请问如何判断是否是数值？

In [28]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype = 'string').str.isnumeric()  
# 检查字符串的每一位字符是否都是由数字组成，小数点也不可以

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

##  六、问题与练习

#### 1. 问题
#####  【问题一】 str对象方法和df/Series对象方法有什么区别？

str对象方法常用于 单元格中的字符串 （即具体某个数据），而df/Series对象方法常用的是对某列的聚合函数，如sum()、mean()等。

##### 【问题二】 给出一列string类型，如何判断单元格是否是数值型数据？

In [30]:
pd.Series(['1.2','-0.3','5','17',np.nan],dtype='string')

0     1.2
1    -0.3
2       5
3      17
4    <NA>
dtype: string

In [32]:
pd.Series(['1.2','-0.3','5','17',np.nan],dtype='string').str.isnumeric()  # 判断是否只包含数字

0    False
1    False
2     True
3     True
4     <NA>
dtype: boolean

In [34]:
pd.Series(['1.2','-0.3','5','17',np.nan],dtype='string').str.extract(r'(\d)') #不行

,0
0,1
1,0
2,5
3,1
4,<NA>


##### 尝试使用str.contains()和正则，但是需要考虑到多种情况

In [43]:
pd.Series(['1.2','-0.3','5','17','1+3',np.nan],dtype='string').str.contains(r'^-?(\d+)(\.\d+)?$')

0     True
1     True
2     True
3     True
4    False
5     <NA>
dtype: boolean

##### 【问题三】 rsplit方法的作用是什么？它在什么场合下适用？

##### 类似于python中的re.split()，使用正则表达式来分割  返回的也是列表

In [50]:
pd.Series(['1+2+3+4','a+b+c+d'],dtype='string').str.rsplit(r'+')

0    [1, 2, 3, 4]
1    [a, b, c, d]
dtype: object

In [51]:
pd.Series(['1+2+3+4','a+b+c+d'],dtype='string').str.rsplit(r'+',expand = False)   # 默认为False

0    [1, 2, 3, 4]
1    [a, b, c, d]
dtype: object

In [53]:
pd.Series(['1+2+3+4','a+b+c+d'],dtype='string').str.rsplit(r'+',expand = True)  

,0,1,2,3
0,1,2,3,4
1,a,b,c,d


#### 【问题四】 在本章的第二到第四节分别介绍了字符串类型的5类操作，请思考它们各自应用于什么场景？

##### 拆分与拼接
#####   str.split--拆分
#####   str.cat--拼接
##### 替换
##### 子串匹配与提取
##### extract--从pandas对象的一列中提取信息成为新的列,例如将手机号码,邮件从文本中提取出来
##### exractall--提取所有能够匹配到的子串,并建立多级索引
##### contain--判断包含某个特定正则格式的子串
##### match--字符串以某个特定子串开头
##### 过滤与判断

### 2. 练习

##### 【练习一】 现有一份关于字符串的数据集，请解决以下问题：
##### （a）现对字符串编码存储人员信息（在编号后添加ID列），使用如下格式：“×××（名字）：×国人，性别×，生于×年×月×日”
##### （b）将（a）中的人员生日信息部分修改为用中文表示（如一九七四年十月二十三日），其余返回格式不变。
##### （c）将（b）中的ID列结果拆分为原列表相应的5列，并使用equals检验是否一致。

In [61]:
pd.read_csv(r'C:/Users\gan\Desktop\joyful-pandas\data\String_data_one.csv',index_col='人员编号').head()

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24
